In [2]:
pip install numpy==2.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.0.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: Could not install packages due to an OSError: [Errno 16] Device or resource busy: '.nfs65b84c904e40507100000cc3'

Note: you may need to restart the kernel to use updated packages.


In [1]:
# load data from csv
# path for input and target data tables

diagnosis_path = '../../test_phase_2/diagnosis_hadm.csv'
discharge_path ='../../test_phase_2/discharge.csv'
edstays_path = '../../test_phase_2/edstays.csv'
radiology_path = '../../test_phase_2/radiology.csv'
triage_path = '../../test_phase_2/triage.csv'
target_path = '../../test_phase_2/discharge_target.csv'
discharge_sections_path = '../../test_phase_2/discharge_sections.csv'
radiology_sections_path = '../../test_phase_2/radiology_sections.csv'

import pandas as pd
import re
import os

# read data
diagnosis_df = pd.read_csv(diagnosis_path, keep_default_na=False)
discharge_df = pd.read_csv(discharge_path, keep_default_na=False)
edstays_df = pd.read_csv(edstays_path, keep_default_na=False)
radiology_df = pd.read_csv(radiology_path, keep_default_na=False)
triage_df = pd.read_csv(triage_path, keep_default_na=False)
target_df = pd.read_csv(target_path, keep_default_na=False)

discharge_sections_df = pd.read_csv(discharge_sections_path, keep_default_na=False)
radiology_sections_df = pd.read_csv(radiology_sections_path, keep_default_na=False)

/home/tr.caron/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/tr.caron/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
from huggingface_hub import login
login(token=HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/tr.caron/.cache/huggingface/token
Login successful


In [3]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

# Initialize the LLM object with the specified model
model = LLM(model="meta-llama/Meta-Llama-3-8B-Instruct")
# model = LLM(model="meta-llama/Meta-Llama-3-8B")
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3-8B')
# Imports
import torch
# generate_prompt function
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. 
 
### Instruction:
{instruction}

### Input:
{input}

### Response:
"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
{instruction}

### Response:
"""


WARNING 10-03 14:45:01 _custom_ops.py:18] Failed to import from vllm._C with ImportError('/home/tr.caron/.local/lib/python3.9/site-packages/vllm/_C.abi3.so: undefined symbol: cuTensorMapEncodeTiled')
INFO 10-03 14:45:20 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=Non

/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/tr.caron/.local/lib/python3.9/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/tr.caron/.local/lib/python3.9/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
/home/tr.caron/.local/lib/python3.9/site-packages/torch/cu

RuntimeError: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver.

In [4]:
def generate_with_model(instruction, context_input, model):
    # Generate the prompt using the instruction and context
    prompt = generate_prompt(instruction, context_input)

    # Setup sampling parameters for generation
    # Here we only calculate length to control the output length dynamically
    context_length = len(tokenizer(prompt, return_tensors="pt").input_ids[0])
    print(context_length)
    stop_str = "--- end of response"
    max_input_length = min(2048, tokenizer.model_max_length - context_length)
    sampling_params = SamplingParams(temperature=0.01, max_tokens=max_input_length, stop=stop_str,)

    # Generate the response using the model
    outputs = model.generate(prompt, sampling_params)
    response_text = outputs[0].outputs[0].text  # Adjust based on actual output structure

    # outputs = llm.generate(final_prompt, sampling_params)
    # answer_texts = [output.outputs[0].text for output in outputs]
    
    # # Extract the "Response" part from the generated text
    # response = response_text.split("### Response:")[1].strip()
    # response = response_text.split("--- end of response")[1].strip() 
    
    # Print and return the response
    print(response_text)
    return response_text

In [5]:
def generate_section(hadm_id, question, dict_mapping=None, additional_context=''):
    topic = '''
    The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
    from a patient's hospital admission. 
    You will be provided with input text and data from a specific patient's medical records and clinical notes 
    delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
    '''.replace('\n', '').replace('    ', '')

    prompt = f'''\n
    {question}

    --- end of response

    Stop generating when you reach "--- end of response"
    Do not repeat the instructions in your response or make comments after your response.
    '''.replace('    ', '')

    context_str = ''

    # Map string keys to actual DataFrame objects
    dataframes = {
        'discharge_sections_df': discharge_sections_df,
        'diagnosis_df': diagnosis_df
    }

    # Process dict_mapping only if provided
    if dict_mapping:
        for df_name, columns in dict_mapping.items():
            df = dataframes.get(df_name)
            if df is not None:
                cur_df = df[df['hadm_id'] == hadm_id]
                for col in columns:
                    # Join non-null values and ensure it's treated as a string
                    context_data = cur_df[col].dropna().astype(str).str.cat(sep=' ')
                    context_str += f"The {col.replace('_', ' ')} was --- {context_data}\n"

    instruction = f"{topic} \n {prompt}"
    context_input = f"<text> {context_str}</text>"

    # If additional context is provided, append it to the context_input
    if additional_context:
        context_input += f"\n<additional_context> {additional_context} </additional_context>"

    response = generate_with_model(instruction, context_input, model)
    return response


In [ ]:
def generate_all_sections(hadm_id):
    a_reason_for_admission = generate_section(hadm_id, "Given the HPI, chief complaint, and initial ICD diagnosis, Why was the patient admitted to the hospital? What symptoms, medical condition, or other reason caused the patient to come to or be brought into the hospital?", dict_mapping={
        'discharge_sections_df': ['CC', 'HPI', 'Major Surgical Procedure'],
        'diagnosis_df': ['icd_title']})
    a_relevant_pmh = generate_section(hadm_id, "Given the History of Present Illness, summarize the patient's major medical conditions and the reason they were admitted to the hospital.", dict_mapping={
        'discharge_sections_df': ['HPI']})
    a_acute_problems = generate_section(hadm_id, "Given the history of present illness, admission and discharge diagnosis, What medical conditions or symptoms was this patient treated for in the hospital?", dict_mapping={
        'discharge_sections_df': ['HPI', 'Discharge Diagnosis'],
        'diagnosis_df': ['icd_title']})
    a_initial_treatments = generate_section(hadm_id, "Given the history of present illness, summarize the treatments and diagnostic workup.", dict_mapping={
        'discharge_sections_df': ['HPI']})
    a_initial_workup = generate_section(hadm_id, "Given the History of Present illness, What was the initial diagnostic work up and pertinent results?", dict_mapping={
        'discharge_sections_df': ['HPI']})
    a_confirmed_dx = generate_section(hadm_id, "What medical conditions were confirmed with diagnostic workup including radiology notes, laboratory, microbiology culture, and other studies?", dict_mapping={
        'discharge_sections_df': ['HPI', 'Pertinent Results']})
    #double check below
    a_ruled_out_dx = generate_section(hadm_id, "What medical conditions were ruled out with diagnostic workup including radiology notes, laboratory results, microbiology culture, and other studies?", dict_mapping={
        'discharge_sections_df': ['Pertinent Results'],
        'radiology_sections_df': ['EXAMINATION', 'INDICATION', 'TECHNIQUE', 'COMPARISON', 'FINDINGS', 'IMPRESSION']})
    a_ongoing_problems = generate_section(hadm_id, "Based on the admission medication list, what other existing or ongoing medical conditions did the patient have?", dict_mapping={
        'discharge_sections_df': ['Medication Lists']})
    a_studies = generate_section(hadm_id, "Given the radiology notes, What radiology, imaging or other studies that were performed and for what inducation? List the radiological and imaging studies. Summarize the type of study, the indication, and impressions from each study.", dict_mapping={
        'radiology_sections_df': ['EXAMINATION', 'INDICATION', 'TECHNIQUE', 'IMPRESSION'],
        'discharge_sections_df': ['Pertinent Results']})
    a_lab_results = generate_section(hadm_id, "Given the medical conditions and lab work, what are the pertinent positives and negatives relating to the patient's symptoms and diagnostic workup", dict_mapping={
        'discharge_sections_df': ['Pertinent Results']}, 
                                    additional_context=a_acute_problems)
    a_micro_results = generate_section(hadm_id, "Given the pertenent results, were there any blood culture  or microbiology studiess?  What were the results?  Did they grow anything? If so, what was the organism and what antibiotics was it sensitive to?", dict_mapping={
        'discharge_sections_df': ['Pertinent Results']})
    #double check the two variables below
    a_differential_dx = generate_section(hadm_id, "Given the HPI, what is the differential diagnosis for the presenting symptoms or chief complaint?", dict_mapping={
        'discharge_sections_df': ['CC', 'HPI']})
    a_rec_work_up = generate_section(hadm_id, "What is the recommended diagnostic work up?", dict_mapping={
        'discharge_sections_df': ['CC', 'HPI']})
    a_med_changes = generate_section(hadm_id, "Given the admission and discharge medication lists if they are present in the note, What were the changes to the patient's medications? What medications were new?  What medications were stopped?", dict_mapping={
        'discharge_sections_df': ['Medication Lists']}) 
    a_med_tx = generate_section(hadm_id, "Given the history of present illness, What medications or IV therapies were used to treat the patient during this hospital admission?", dict_mapping={
        'discharge_sections_df': ['HPI']},
                                additional_context=a_med_changes)
    a_major_procedures = generate_section(hadm_id, "What were the major surgical, invasive, or diagnostic, or therapeutic procedures?", dict_mapping={
        'discharge_sections_df': ['Major Surgical Procedure']},
                                         additional_context=a_initial_treatments)
    a_hospital_problems = generate_section(hadm_id, "What were the acute and chronic medical conditions the patient was treated for in the hospital?", 
                                           additional_context=(a_acute_problems + "\n" + a_ongoing_problems))
    a_presentation_symptoms = generate_section(hadm_id, "For each acute medical condition, what symptoms did the patient present with? What was the severity?", dict_mapping={
        'discharge_sections_df': ['HPI']}, additional_context=a_acute_problems)
    a_diagnostic_work_up = generate_section(hadm_id, "For each acute medical condition, what was the diagnostic work up and related results?", 
                                            additional_context=(a_hospital_problems + "\n" + a_initial_workup))
    a_treatment_course = generate_section(hadm_id, "For each acute medical condition, what were the treatments?",
                                          additional_context=(a_acute_problems + "\n" + a_med_tx + "\n" + a_initial_treatments
                                                             + "\n" + a_med_changes + "\n" + a_major_procedures))
    a_provider_instructions = generate_section(hadm_id, "Given the Transitional issues, What are the instructions for the recieving provider?", dict_mapping={
        'discharge_sections_df': ['Transitional Issues']})
    a_follow_up = generate_section(hadm_id, "Given the Transitional issues, what is the recommended follow up?", dict_mapping={
        'discharge_sections_df': ['Transitional Issues']})
    a_pending_results = generate_section(hadm_id, "Given the Transitional issues, are there pending results?", dict_mapping={
        'discharge_sections_df': ['Transitional Issues']})
    a_dx_list = generate_section(hadm_id, "What diagnosis did the patient recieve? Cluster these with the acute medical conditions that were identified.", dict_mapping={
        'discharge_sections_df': ['Discharge Diagnosis'],
        'diagnosis_df': ['icd_code', 'icd_version', 'icd_title']}, additional_context=a_acute_problems)
    a_physical_exam = generate_section(hadm_id, "What were the pertinent physical exam findings and notable vital signs related to the acute medical conditions?", dict_mapping={
        'discharge_sections_df': ['Physical Exam']})
    

    
    

In [6]:
bhc_df = pd.DataFrame(columns=[
    'hadm_id', 'initial_treatments', 'major_procedures', 'med_changes', 'med_tx', 
    'initial_workup', 'reason_for_admission', 'relevant_pmh', 'acute_problems', 
    'ongoing_problems', 'hospital_problems', 'presentation_symptoms', 
    'diagnostic_work_up', 'treatment_course', 'rec_work_up', 'pending_results', 
    'bhc_summary'])

In [7]:
bhc_df_5 = pd.DataFrame(columns=[
    'hadm_id', 'initial_treatments', 'major_procedures', 'med_changes', 'med_tx', 
    'initial_workup', 'reason_for_admission', 'relevant_pmh', 'acute_problems', 
    'ongoing_problems', 'hospital_problems', 'presentation_symptoms', 
    'diagnostic_work_up', 'treatment_course', 'rec_work_up', 'pending_results', 
    'bhc_summary'])

In [8]:
def generate_bhc(hadm_id):
    a_initial_treatments = generate_section(hadm_id, "Given the history of present illness, summarize the treatments and diagnostic workup.", dict_mapping={
        'discharge_sections_df': ['HPI']})
    a_major_procedures = generate_section(hadm_id, "What were the major surgical, invasive, or diagnostic, or therapeutic procedures?", dict_mapping={
        'discharge_sections_df': ['Major Surgical Procedure']},
                                         additional_context=a_initial_treatments)
    a_med_changes = generate_section(hadm_id, "Given the admission and discharge medication lists if they are present in the note, What were the changes to the patient's medications? What medications were new?  What medications were stopped?", dict_mapping={
        'discharge_sections_df': ['Medication Lists']}) 
    a_med_tx = generate_section(hadm_id, "Given the history of present illness, What medications or IV therapies were used to treat the patient during this hospital admission?", dict_mapping={
        'discharge_sections_df': ['HPI']},
                                additional_context=a_med_changes)
    a_initial_workup = generate_section(hadm_id, "Given the History of Present illness, What was the initial diagnostic work up and pertinent results?", dict_mapping={
        'discharge_sections_df': ['HPI']})
    a_reason_for_admission = generate_section(hadm_id, "Given the HPI, chief complaint, and initial ICD diagnosis, Why was the patient admitted to the hospital? What symptoms, medical condition, or other reason caused the patient to come to or be brought into the hospital?", dict_mapping={
        'discharge_sections_df': ['CC', 'HPI', 'Major Surgical Procedure'],
        'diagnosis_df': ['icd_title']})
    a_relevant_pmh = generate_section(hadm_id, "Given the History of Present Illness, summarize the patient's major medical conditions and the reason they were admitted to the hospital.", dict_mapping={
        'discharge_sections_df': ['HPI']})
    a_acute_problems = generate_section(hadm_id, "Given the history of present illness, admission and discharge diagnosis, What medical conditions or symptoms was this patient treated for in the hospital?", dict_mapping={
        'discharge_sections_df': ['HPI', 'Discharge Diagnosis'],
        'diagnosis_df': ['icd_title']})
    a_ongoing_problems = generate_section(hadm_id, "Based on the admission medication list, what other existing or ongoing medical conditions did the patient have?", dict_mapping={
        'discharge_sections_df': ['Medication Lists']})
    a_hospital_problems = generate_section(hadm_id, "What were the acute and chronic medical conditions the patient was treated for in the hospital?", 
                                           additional_context=(a_acute_problems + "\n" + a_ongoing_problems))
    a_presentation_symptoms = generate_section(hadm_id, "For each acute medical condition, what symptoms did the patient present with? What was the severity?", dict_mapping={
        'discharge_sections_df': ['HPI']}, additional_context=a_acute_problems)
    a_diagnostic_work_up = generate_section(hadm_id, "For each acute medical condition, what was the diagnostic work up and related results?", 
                                            additional_context=(a_hospital_problems + "\n" + a_initial_workup))
    a_treatment_course = generate_section(hadm_id, "For each acute medical condition, what were the treatments?",
                                          additional_context=(a_acute_problems + "\n" + a_med_tx + "\n" + a_initial_treatments
                                                             + "\n" + a_med_changes + "\n" + a_major_procedures))
    a_rec_work_up = generate_section(hadm_id, "What is the recommended diagnostic work up?", dict_mapping={
        'discharge_sections_df': ['CC', 'HPI']})
    a_pending_results = generate_section(hadm_id, "Given the Transitional issues, are there pending results?", dict_mapping={
        'discharge_sections_df': ['Transitional Issues']})
    
    context = (a_reason_for_admission + "\n" + a_relevant_pmh + "\n" + a_acute_problems + "\n" + a_presentation_symptoms + "\n" + a_diagnostic_work_up
               + "\n" + a_treatment_course + "\n" + a_rec_work_up + "\n" + a_pending_results)
    bhc = generate_section(hadm_id, "Given the information from the patient's electronic health record about their hospital admission: Create a summary of the events and treatments during the hospital visit.  Start with a summary sentence describing the patient’s relevant past medical history, the reason for the hospital admission, what service they were admitted to, and any major procedure. Then summarize for each acute medical condition, the presenting symptom and severity, the differential diagnosis, related diagnostic workup and results, related treatments or procedures, the relevant physical exam findings, pertinent lab work, radiology, imaging, cardiac studies.  Are there pending results?  Is there recommended follow up related to this medical condition? What medications did they receive related to each medical condition?", 
                           additional_context = context)
    
    new_row = {
        'hadm_id': hadm_id,
        'initial_treatments': a_initial_treatments,
        'major_procedures': a_major_procedures,
        'med_changes': a_med_changes,
        'med_tx': a_med_tx,
        'initial_workup': a_initial_workup,
        'reason_for_admission': a_reason_for_admission,
        'relevant_pmh': a_relevant_pmh,
        'acute_problems': a_acute_problems,
        'ongoing_problems': a_ongoing_problems,
        'hospital_problems': a_hospital_problems,
        'presentation_symptoms': a_presentation_symptoms,
        'diagnostic_work_up': a_diagnostic_work_up,
        'treatment_course': a_treatment_course,
        'rec_work_up': a_rec_work_up,
        'pending_results': a_pending_results,
        'bhc_summary': bhc
    }

    # append new row to the global dataframe
    global bhc_df_5
    bhc_df_5 = pd.concat([bhc_df_5, pd.DataFrame([new_row])], ignore_index=True)
    
    return bhc

    
    

In [9]:
def generate_di(hadm_id):
    a_initial_treatments = generate_section(hadm_id, "Given the history of present illness, summarize the treatments and diagnostic workup.", dict_mapping={
        'discharge_sections_df': ['HPI']})
    a_major_procedures = generate_section(hadm_id, "What were the major surgical, invasive, or diagnostic, or therapeutic procedures?", dict_mapping={
        'discharge_sections_df': ['Major Surgical Procedure']},
                                         additional_context=a_initial_treatments)
    a_med_changes = generate_section(hadm_id, "Given the admission and discharge medication lists if they are present in the note, What were the changes to the patient's medications? What medications were new?  What medications were stopped?", dict_mapping={
        'discharge_sections_df': ['Medication Lists']})
    a_med_tx = generate_section(hadm_id, "Given the history of present illness, What medications or IV therapies were used to treat the patient during this hospital admission?", dict_mapping={
        'discharge_sections_df': ['HPI']},
                                additional_context=a_med_changes)
    a_acute_problems = generate_section(hadm_id, "Given the history of present illness, admission and discharge diagnosis, What medical conditions or symptoms was this patient treated for in the hospital?", dict_mapping={
        'discharge_sections_df': ['HPI', 'Discharge Diagnosis'],
        'diagnosis_df': ['icd_title']})
    a_reason_for_admission = generate_section(hadm_id, "Given the HPI, chief complaint, and initial ICD diagnosis, Why was the patient admitted to the hospital? What symptoms, medical condition, or other reason caused the patient to come to or be brought into the hospital?", dict_mapping={
        'discharge_sections_df': ['CC', 'HPI', 'Major Surgical Procedure'],
        'diagnosis_df': ['icd_title']})
    a_treatment_course = generate_section(hadm_id, "For each acute medical condition, what were the treatments?",
                                          additional_context=(a_acute_problems + "\n" + a_med_tx + "\n" + a_initial_treatments
                                                             + "\n" + a_med_changes + "\n" + a_major_procedures))
    a_provider_instructions = generate_section(hadm_id, "Given the Transitional issues, What are the instructions for the recieving provider?", dict_mapping={
        'discharge_sections_df': ['Transitional Issues']})
    a_med_changes = generate_section(hadm_id, "Given the admission and discharge medication lists if they are present in the note, What were the changes to the patient's medications? What medications were new?  What medications were stopped?", dict_mapping={
        'discharge_sections_df': ['Medication Lists']}) 
    
    context = (a_reason_for_admission + "\n" + a_treatment_course + "\n" + a_provider_instructions + "\n" + a_med_changes)
    di = generate_section(hadm_id, "Given the information from the patient's EHR about their hospital admission, Compose a letter to the patient that is courteous and easy to understand.  There should be limited medical jargon and it should be written in layman's language.  The letter will describe briefly the reason for admission and what treatments were given.  It will also include any major changes to the patient's current medical management. Include the following information: why the patient was admitted to the hospital, including major medical condition or symptoms. What were the most relevant diagnostic tests and what did they show? What major treatments did the patient receive? Are there any pending results?  What are the changes to the existing medications? Is there any scheduled or recommended  follow up? Limit the result to 600 words.", 
                           additional_context = context)
    return di

    
    

In [10]:
pwd

'/work/AI-EHR/discharge_me/data/Experiments/CoT Prompting'

In [11]:
hadm_subset_df = pd.read_csv('/work/AI-EHR/discharge_me/data/Experiments/hadm_id_for_discharge_me_scoring.csv')

In [21]:
pip install tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# FOR 5 HADM_IDS
from tqdm import tqdm
import pandas as pd
import time

golden_hadm_ids = [22007257, 22289537, 22988214, 25498079, 22887667]

# Create an empty results table
results = pd.DataFrame(columns=['hadm_id', 'BHC_LLM', 'BHC_human'])

# results path:
results_path = '../../troyTest/'
file_name = '5_CoT_bhc_results.csv'
results_file_path = os.path.join(results_path, file_name)

count = 0
total_records = len(golden_hadm_ids)
for hadm_id in tqdm(golden_hadm_ids):  # Replace with your actual hadm_id values
    count += 1
    percent_complete = count*100/total_records
    print(f"{count} of {total_records}, {percent_complete}% hadm_id: {hadm_id} **********\n\n")
    start_time_bhc = time.time()
    BHC_LLM = generate_bhc(hadm_id)
    end_time_bhc = time.time()
    duration_bhc = end_time_bhc - start_time_bhc
    print(f"Time taken for generate_bhc: {duration_bhc} seconds.")
    # discharge_instructions = generate_discharge_instructions(hadm_id)
    new_row = pd.DataFrame({'hadm_id': [hadm_id], 'BHC_LLM': [BHC_LLM], 'BHC_human': [discharge_sections_df[discharge_sections_df['hadm_id'] == hadm_id]['BHC'].astype(str)]})
    results = pd.concat([results, new_row], ignore_index=True)
    # results = results.append({'hadm_id': hadm_id, 'brief_hospital_course': brief_hospital_course, 'discharge_instructions': discharge_instructions}, ignore_index=True)
    results.to_csv(results_file_path, index=False)

  0%|          | 0/5 [00:00<?, ?it/s]

1 of 5, 20.0% hadm_id: 22007257 **********


799



Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.85s/it, est. speed input: 164.96 toks/s, output: 80.72 toks/s]


The patient, a 65-year-old retired woman with a history of coronary artery disease, hypertension, and non-insulin-dependent diabetes mellitus, presented with chest pain that woke her from sleep. The pain was relieved by nitroglycerin and described as left-sided with associated bilateral chest tightness, dyspnea, sweating, and hypertension. The patient had similar symptoms previously, which resolved. She was hospitalized previously for worsening exertional angina and underwent a pharmacologic nuclear stress test, which showed a reversible, medium-sized, mild perfusion defect involving the left anterior descending territory. The patient refused cardiac catheterization at that time due to a previous experience of a long recovery period. 

The patient's treatments included ASA 324mg, 2mg IV morphine, and heparin gtt. The diagnostic workup included laboratory tests, which were significant for troponin <0.01, and an electrocardiogram that showed lateral ST depressions with sl V2/V3 depressio


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.30it/s, est. speed input: 1356.39 toks/s, output: 73.94 toks/s]


...


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.40s/it, est. speed input: 400.88 toks/s, output: 80.32 toks/s]


The recommended diagnostic work-up for this patient includes a urinalysis, urine culture, and a CT scan with contrast to further evaluate the patient's symptoms and rule out any potential complications. Additionally, a complete blood count (CBC) and blood cultures may be ordered to assess for signs of sepsis. A renal ultrasound may also be considered to evaluate for any potential kidney damage or obstruction. Furthermore, a thorough review of the patient's medication list and medical history may be necessary to identify any potential contributing factors to the patient's condition. 


205



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.65it/s, est. speed input: 751.47 toks/s, output: 76.98 toks/s]


Yes, there are pending results. The blood and urine cultures are pending at the time of discharge.
1337



Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [17]:
bhc_df_5.to_csv("bhc_variables_df_5.csv",index=False)
bhc_df_5

,hadm_id,initial_treatments,major_procedures,med_changes,med_tx,initial_workup,reason_for_admission,relevant_pmh,acute_problems,ongoing_problems,hospital_problems,presentation_symptoms,diagnostic_work_up,treatment_course,rec_work_up,pending_results,bhc_summary
0,22007257,"The patient, a 65-year-old retired woman with ...","The major surgical, invasive, or diagnostic, o...",The changes to the patient's medications were ...,"During this hospital admission, the patient wa...",The initial diagnostic workup included cardiac...,The patient was admitted to the hospital due t...,Ms. ___ was admitted to the hospital due to ch...,The patient was treated for Non-ST Elevation M...,The patient had existing or ongoing medical co...,The patient was treated for acute Non-ST Eleva...,The patient presented with chest pain that wok...,The diagnostic workup for acute Non-ST Elevati...,For the patient's acute medical condition of N...,The recommended diagnostic work-up for this pa...,"Yes, there are pending results. The CBC (Compl...",Ms. ___ was admitted to the hospital due to ch...
1,22289537,"The patient, a 65-year-old female with a histo...",The patient did not undergo any major surgical...,The changes to the patient's medications were ...,"During this hospital admission, the patient wa...",The initial diagnostic workup included a CT ch...,The patient was admitted to the hospital due t...,Ms. ___ was admitted to the hospital due to a ...,"The patient was treated for rib fractures, spe...","Based on the admission medication list, the pa...",The patient was treated for acute rib fracture...,The patient presented with symptoms of lighthe...,The diagnostic workup for the acute rib fractu...,The patient was treated for rib fractures with...,The recommended diagnostic work-up for this pa...,"Yes, there are pending results. The patient ne...",Ms. ___ was admitted to the hospital due to a ...
2,22988214,The patient was treated with an ace bandage to...,"The major surgical, invasive, or diagnostic, o...",The changes to the patient's medications were ...,"During this hospital admission, the patient wa...",The initial diagnostic workup and pertinent re...,The patient was admitted to the hospital due t...,The patient's major medical conditions include...,The patient was treated for a non-traumatic he...,The patient had existing or ongoing medical co...,The patient was treated for acute non-traumati...,The patient presented with symptoms of left le...,The patient had the following acute medical co...,"For each acute medical condition, the treatmen...",The recommended diagnostic work-up for this pa...,"Yes, there are pending results, specifically a...",The patient has a relevant past medical histor...
3,25498079,The patient was treated with a CT scan and adm...,"The major surgical, invasive, or diagnostic, o...",The changes to the patient's medications were ...,The patient did not receive any new medication...,The initial diagnostic work up included liver ...,The patient was admitted to the hospital due t...,The patient's major medical conditions include...,The patient was treated for a Pruritic Disorde...,The patient had existing or ongoing medical co...,The patient was treated for an Ampullary Mass ...,The patient presented with pruritus and dark u...,The diagnostic work-up for the acute medical c...,The patient was treated for a Pruritic Disorde...,The recommended diagnostic work-up is a CT sca...,"Yes, there are pending results. The pathology ...",The patient was admitted to the hospital due t...
4,22887667,The patient was treated with ciprofloxicin 400...,The patient did not undergo any major surgical...,The changes to the patient's medications were ...,"During this hospital admission, the patient wa...",The initial diagnostic workup included a urina...,The patient was admitted to the hospital due t...,The patient's major medical conditions include...,The patient was treated for urinary tract infe...,"The patient had hypertension, as evidenced by ...",

In [54]:
import pandas as pd
import time

# Create an empty results table
results = pd.DataFrame(columns=['hadm_id', 'brief_hospital_course'])

# results path:
results_path = '../../troyTest/'
file_name = 'CoT_results_bhc.csv'
results_file_path = os.path.join(results_path, file_name)

count = 0
total_records = len(target_df.hadm_id)
for hadm_id in target_df['hadm_id'][5:13]:  # Replace with your actual hadm_id values
    count += 1
    percent_complete = count*100/total_records
    print(f"{count} of {total_records}, {percent_complete}% hadm_id: {hadm_id} **********\n\n")
    start_time_bhc = time.time()
    brief_hospital_course = generate_bhc(hadm_id)
    end_time_bhc = time.time()
    duration_bhc = end_time_bhc - start_time_bhc
    print(f"Time taken for generate_bhc: {duration_bhc} seconds.")
    # discharge_instructions = generate_discharge_instructions(hadm_id)
    new_row = pd.DataFrame({'hadm_id': [hadm_id], 'brief_hospital_course': [brief_hospital_course]})
    results = pd.concat([results, new_row], ignore_index=True)
    # results = results.append({'hadm_id': hadm_id, 'brief_hospital_course': brief_hospital_course, 'discharge_instructions': discharge_instructions}, ignore_index=True)
    results.to_csv(results_file_path, index=False)

1 of 10962, 0.009122422915526365% hadm_id: 28723315 **********


631



Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.64s/it, est. speed input: 239.56 toks/s, output: 81.24 toks/s]


The patient was treated with a stress echo test, which revealed inducible ischemia and severe aortic insufficiency. The diagnostic workup included an echocardiography, which showed 4+ aortic regurgitation, and a CTA chest, which was negative for aortic dissection. The patient was admitted to the cardiology service for further management. The patient's symptoms were managed with treatment for his new diagnosis. The patient's review of systems was negative for any prior history of deep venous thrombosis, pulmonary embolism, bleeding at the time of surgery, cough, hemoptysis, black stools or red stools. The patient denied recent fevers, chills or rigors, and denied exertional buttock or calf pain. All other review of systems were negative. The patient's cardiac review of systems was notable for absence of dyspnea on exertion, paroxysmal nocturnal dyspnea, orthopnea, ankle edema, palpitations, syncope or presyncope. 
395



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.54it/s, est. speed input: 608.10 toks/s, output: 78.51 toks/s]


The patient did not undergo any major surgical, invasive, or diagnostic, or therapeutic procedures. However, the patient did undergo a stress echo test, echocardiography, and a CTA chest as part of their diagnostic workup. 
1004


...

The recommended diagnostic work-up for this patient includes a comprehensive evaluation for acute respiratory failure, including a thorough physical examination, chest radiograph, and laboratory tests such as complete blood count, electrolyte panel, and blood gas analysis. Additionally, a thorough review of the patient's medication list, including the ACE inhibitor, is necessary to assess potential contributing factors to her respiratory failure. A sleep study may also be considered to evaluate for obstructive sleep apnea. Furthermore, a thorough neurological evaluation is necessary to assess the patient's mental status and potential contributing factors to her confusion. A thorough review of the patient's medical history, including her history of AVM and seizure disorder, is also necessary to assess potential contributing factors to her respiratory failure. 


207



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.90it/s, est. speed input: 601.09 toks/s, output: 78.40 toks/s]


Yes, there are pending results, specifically the flow cytometry test to rule out T cell lymphoma and histiocytosis.
1555



Processed prompts: 100%|██████████| 1/1 [00:16<00:00, 16.93s/it, est. speed input: 91.85 toks/s, output: 80.93 toks/s]

The patient has a relevant past medical history of multiple AVM and a seizure disorder. She was admitted to the hospital due to symptoms of confusion, shortness of breath, and hypoxia, which led to her admission to the emergency room. The patient was treated for acute exacerbation of COPD, acute diastolic heart failure, and respiratory failure in the hospital. She also had underlying medical conditions such as epilepsy, left frontal AVM, posterior communicating artery aneurysm, lumbar disc herniation, and hypertension. The patient's symptoms included shortness of breath, hypoxia, and altered mental status. She was also experiencing cough productive of gray sputum and had a history of seizure disorder.

The patient presented with symptoms of shortness of breath, hypoxia, and altered mental status. The severity of these symptoms was moderate to severe, as evidenced by her need for oxygen therapy and BiPAP in the hospital. She also had a cough productive of gray sputum, which was likely r

In [56]:
import pandas as pd
import time

# Create an empty results table
results = pd.DataFrame(columns=['hadm_id', 'discharge_instructions'])

# results path:
results_path = '../../troyTest/'
file_name = 'CoT_results_di.csv'
results_file_path = os.path.join(results_path, file_name)

count = 0
total_records = len(target_df.hadm_id)
for hadm_id in target_df['hadm_id'][5:13]:  # Replace with your actual hadm_id values
    count += 1
    percent_complete = count*100/total_records
    print(f"{count} of {total_records}, {percent_complete}% hadm_id: {hadm_id} **********\n\n")
    start_time_di = time.time()
    discharge_instructions = generate_di(hadm_id)
    end_time_di = time.time()
    duration_di = end_time_di - start_time_di
    print(f"Time taken for generate_di: {duration_di} seconds.")
    # discharge_instructions = generate_discharge_instructions(hadm_id)
    new_row = pd.DataFrame({'hadm_id': [hadm_id], 'discharge_instructions': [discharge_instructions]})
    results = pd.concat([results, new_row], ignore_index=True)
    # results = results.append({'hadm_id': hadm_id, 'brief_hospital_course': brief_hospital_course, 'discharge_instructions': discharge_instructions}, ignore_index=True)
    results.to_csv(results_file_path, index=False)

1 of 10962, 0.009122422915526365% hadm_id: 28723315 **********


631



Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.64s/it, est. speed input: 239.26 toks/s, output: 81.14 toks/s]


The patient was treated with a stress echo test, which revealed inducible ischemia and severe aortic insufficiency. The diagnostic workup included an echocardiography, which showed 4+ aortic regurgitation, and a CTA chest, which was negative for aortic dissection. The patient was admitted to the cardiology service for further management. The patient's symptoms were managed with treatment for his new diagnosis. The patient's review of systems was negative for any prior history of deep venous thrombosis, pulmonary embolism, bleeding at the time of surgery, cough, hemoptysis, black stools or red stools. The patient denied recent fevers, chills or rigors, and denied exertional buttock or calf pain. All other review of systems were negative. The patient's cardiac review of systems was notable for absence of dyspnea on exertion, paroxysmal nocturnal dyspnea, orthopnea, ankle edema, palpitations, syncope or presyncope. 
395



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.53it/s, est. speed input: 606.94 toks/s, output: 78.36 toks/s]


The patient did not undergo any major surgical, invasive, or diagnostic, or therapeutic procedures. However, the patient did undergo a stress echo test, echocardiography, and a CTA chest as part of their diagnostic workup. 
1004


...

The instructions for the receiving provider are to evaluate the patient's pruritis, rule out T cell lymphoma and histiocytosis with flow cytometry, and restart Lasix as needed to manage volume status.
475



Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.93s/it, est. speed input: 245.76 toks/s, output: 81.75 toks/s]


The changes to the patient's medications were the addition of ProAir HFA (albuterol sulfate), Symbicort (budesonide-formoterol), Amlodipine, and DiphenhydrAMINE, and the addition of Sarna Lotion. The medications that were stopped were Lorazepam and Lisinopril. The medications that were new were ProAir HFA (albuterol sulfate), Symbicort (budesonide-formoterol), Amlodipine, DiphenhydrAMINE, and Sarna Lotion. The medications that were continued were LaMOTrigine, LeVETiracetam, PHENObarbital, and Phenytoin Sodium Extended.  
906



Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.84s/it, est. speed input: 155.17 toks/s, output: 81.01 toks/s]

Dear [Patient's Name],

I am writing to inform you about your recent hospital admission and the treatments you received during your stay. You were admitted to the hospital due to symptoms of confusion, shortness of breath, and hypoxia. Our medical team worked hard to address these symptoms and provide you with the best possible care.

During your admission, we performed several diagnostic tests to determine the cause of your symptoms. These tests included bronchoscopy and lymph node biopsy, which helped us to identify the presence of an acute exacerbation of COPD, acute diastolic heart failure, respiratory failure, and epilepsy. We also found that you have a left frontal AVM, posterior communicating artery aneurysm, lumbar disc herniation, hypertension, sclerotic lesions in the T11 vertebral body and manubrium, and moderate mediastinal and hilar lymphadenopathy.

To manage your symptoms and conditions, we provided you with several treatments. These included IV lasix, oxygen therapy, Bi